1. Import Libraries

In [ ]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from github import Github

2. Load Env variables

In [ ]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
REPO_NAME = os.getenv("REPO_NAME")
MAIN_BRANCH = os.getenv("MAIN_BRANCH", "main")

REPO_NAME

3. configure the LLM

In [ ]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.0,
    max_retries=2,
)

# llm.invoke("who is the pm of india")

4. Create diagnosis agent

In [ ]:
def diagnosis_agent(error_text: str, main_tf: str) -> str:
    prompt = f"""
You are a Terraform diagnosis and fixing agent.

Here is the error log:
----------------------
{error_text}

Here is the main.tf file:
-------------------------
{main_tf}

Your job:
1. Understand the error.
2. Produce the corrected main.tf code ONLY.
3. Do NOT explain — output only the fixed main.tf contents in plain Terraform code.
4. Do NOT wrap the code in markdown code blocks. Return raw Terraform code only.

Return ONLY valid Terraform code for main.tf without any markdown formatting.
"""
    try:
        # LLM call — use invoke API
        resp = llm.invoke([HumanMessage(content=prompt)])
        print("✅ LLM Response received")
        
        # Extract content from response
        fixed_text = None
        if hasattr(resp, "content"):
            fixed_text = resp.content
        elif isinstance(resp, dict) and "content" in resp:
            fixed_text = resp["content"]
        else:
            fixed_text = str(resp)
        
        # Clean up markdown code blocks if present
        if fixed_text.startswith("```"):
            # Remove ```terraform or ``` markers
            fixed_text = fixed_text.split("```")[1]
            if fixed_text.startswith("terraform"):
                fixed_text = fixed_text[10:]  # Remove "terraform"
            fixed_text = fixed_text.strip()
        
        fixed_code = fixed_text.strip()
        print(f"✅ Fixed code extracted ({len(fixed_code)} chars)")
        return fixed_code
    except Exception as e:
        print(f"❌ Error in diagnosis_agent: {e}")
        raise

5. load the error text file and main.tf (errors)

In [ ]:
error_file_path = r"C:\Users\virub\OneDrive\Desktop\Hackathon sat test\Errors\1. Terraform errors.txt"
main_tf_path = r"C:\Users\virub\OneDrive\Desktop\Hackathon sat test\main.tf"

with open(error_file_path, "r", encoding='utf-8') as f:
    error_text = f.read()

with open(main_tf_path, "r", encoding='utf-8') as f:
    main_tf = f.read()

# print(error_text)
# print(main_tf)

In [ ]:
fixed_terraform_code = diagnosis_agent(error_text, main_tf)

6. store fixed code into fixed_main.tf file

In [ ]:
# Save it as a .tf file
with open("fixed_main.tf", "w") as tf_file:
    tf_file.write(fixed_terraform_code)

7. Create github_agent (create new branch and commit the fixed_main.tf file)

In [ ]:
github_file_path = "Function APP/infra/main.tf"
new_branch_name = f"raju_{uuid.uuid4().hex[:4]}"

def create_github_agent():
    # Ask for human approval first
    approval = input("Do you approve creating a new branch and committing the fix? (yes/no): ").strip().lower()
    
    if approval not in ["yes", "y"]:
        print("❌ Human rejected approval. Exiting without changes.")
        return  # Exit the function

    # Connect to GitHub
    gh = Github(GITHUB_TOKEN)
    print(f"Connected as: {gh.get_user().login}")
    
    repo = gh.get_repo(REPO_NAME)
    print(f"Repository accessed: {repo.full_name}")

    # Get main branch ref
    base_ref = repo.get_git_ref(f"heads/{MAIN_BRANCH}")
    print(f"Base ref: {base_ref.ref}")

    # Create a new branch from main
    import uuid
    new_branch_name = f"raju_{uuid.uuid4().hex[:4]}"
    repo.create_git_ref(ref=f"refs/heads/{new_branch_name}", sha=base_ref.object.sha)
    print(f"✅ Branch created: {new_branch_name}")

    # Commit the latest fixed_main.tf to the new branch
    file_path = github_file_path
    contents = repo.get_contents(file_path, ref=new_branch_name)
    repo.update_file(
        path=file_path,
        message="🤖 AI: Fix Terraform errors (automated)",
        content=fixed_terraform_code,
        sha=contents.sha,
        branch=new_branch_name,
    )
    print(f"✅ Updated: {file_path}")
    print(f"✅ Committed to branch: {new_branch_name}")


In [ ]:
create_github_agent()